# Modelos de ML

**Objetivo**: Criar modelos de ML para a projeção de todas as nossas séries.

**Metodologias**: 

- Regressão Linear
- Árvore de decisão
- Random Forest
- XGBoost
- LightGBM

## 0. Setup

In [1]:
%load_ext autotime

time: 112 µs (started: 2024-01-04 14:04:17 -03:00)


In [2]:
#---- Manipulação de dados:

import pandas as pd
import numpy as np

#---- Modelagem:

from hierarchicalforecast.utils import aggregate
from mlforecast import MLForecast
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

#---- Reconciliação

from hierarchicalforecast.methods import BottomUp, TopDown, ERM, OptimalCombination, MinTrace, MiddleOut
from hierarchicalforecast.core import HierarchicalReconciliation

#---- Visualização

import plotly.express as px

time: 1.44 s (started: 2024-01-04 14:04:17 -03:00)


## 1. Dados: vendas de roupas no varejo

In [3]:
dados = pd.read_csv('https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-hierarchical-forecasting/main/retail-usa-clothing.csv')

dados.head()

,date,state,item,quantity,region,country
0,1997-11-25,NewYork,mens_clothing,8,Mid-Alantic,USA
1,1997-11-26,NewYork,mens_clothing,9,Mid-Alantic,USA
2,1997-11-27,NewYork,mens_clothing,11,Mid-Alantic,USA
3,1997-11-28,NewYork,mens_clothing,11,Mid-Alantic,USA
4,1997-11-29,NewYork,mens_clothing,10,Mid-Alantic,USA


time: 958 ms (started: 2024-01-04 14:04:18 -03:00)


In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388024 entries, 0 to 388023
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      388024 non-null  object
 1   state     388024 non-null  object
 2   item      388024 non-null  object
 3   quantity  388024 non-null  int64 
 4   region    388024 non-null  object
 5   country   388024 non-null  object
dtypes: int64(1), object(5)
memory usage: 17.8+ MB
time: 53 ms (started: 2024-01-04 14:04:19 -03:00)


## 2. Modificação nos dados 

In [5]:
def clean_data_baseline(df: pd.DataFrame):

    #---- 1. Excluindo a variável de country:

    df = df\
        .drop(columns = 'country')

    #---- 2. Mudando o tipo da variável de date para datetime:

    df['date'] = pd.to_datetime(df['date'])

    #---- 3. Renomeando as variáveis de quantidade de vendas e data:
    # date -> ds
    # quantity -> y

    df = df\
        .rename(columns = {'date': 'ds', 
                           'quantity': 'y'})

    return df

time: 746 µs (started: 2024-01-04 14:04:19 -03:00)


In [6]:
df = clean_data_baseline(df = dados)

df.head()

,ds,state,item,y,region
0,1997-11-25,NewYork,mens_clothing,8,Mid-Alantic
1,1997-11-26,NewYork,mens_clothing,9,Mid-Alantic
2,1997-11-27,NewYork,mens_clothing,11,Mid-Alantic
3,1997-11-28,NewYork,mens_clothing,11,Mid-Alantic
4,1997-11-29,NewYork,mens_clothing,10,Mid-Alantic


time: 99.1 ms (started: 2024-01-04 14:04:19 -03:00)


In [7]:
def format_hierarchical_df(df: pd.DataFrame, cols_hierarchical: list):

    #---- 1. Cria uma lista de listas: [[col1], [col1, col2], ..., [col1, col2, coln]]

    hier_list = [cols_hierarchical[:i] for i in range(1, len(cols_hierarchical) + 1)]

    #---- 2. Aplica a função aggregate que formata os dados em que a lib hierarchical pede

    Y_df, S_df, tags = aggregate(df = df, spec = hier_list)

    return Y_df, S_df, tags

time: 999 µs (started: 2024-01-04 14:04:19 -03:00)


In [8]:
cols_hierarchical = ['region', 'state', 'item']

Y_df, S_df, tags = format_hierarchical_df(df = df, cols_hierarchical = cols_hierarchical)

time: 564 ms (started: 2024-01-04 14:04:19 -03:00)


In [9]:
display(Y_df.head())
display(Y_df.tail())

,ds,y
unique_id,,
EastNorthCentral,1997-11-25,507
EastNorthCentral,1997-11-26,504
EastNorthCentral,1997-11-27,510
EastNorthCentral,1997-11-28,507
EastNorthCentral,1997-11-29,513


,ds,y
unique_id,,
SouthCentral/Tennessee/womens_shoes,2009-07-24,31
SouthCentral/Tennessee/womens_shoes,2009-07-25,30
SouthCentral/Tennessee/womens_shoes,2009-07-26,31
SouthCentral/Tennessee/womens_shoes,2009-07-27,29
SouthCentral/Tennessee/womens_shoes,2009-07-28,30


time: 8.65 ms (started: 2024-01-04 14:04:20 -03:00)


- **Dados de treino: 25/11/1997 a 31/12/2008**
- **Dados de validação: 01/01/2009 a 28/07/2009**

In [10]:
def split_train_test(df: pd.DataFrame, dt_start_train: str):

    #---- 1. Dados de treino

    train = df.query(f'ds < "{dt_start_train}"')

    #---- 2. Dados de teste:
    
    valid = df.query(f'ds >= "{dt_start_train}"')

    return train, valid

time: 671 µs (started: 2024-01-04 14:04:20 -03:00)


In [11]:
Y_train_df, Y_valid_df = split_train_test(df = Y_df, dt_start_train = '2009-01-01')

time: 78.9 ms (started: 2024-01-04 14:04:20 -03:00)


In [12]:
display(Y_train_df.head())
display(Y_train_df.tail())

,ds,y
unique_id,,
EastNorthCentral,1997-11-25,507
EastNorthCentral,1997-11-26,504
EastNorthCentral,1997-11-27,510
EastNorthCentral,1997-11-28,507
EastNorthCentral,1997-11-29,513


,ds,y
unique_id,,
SouthCentral/Tennessee/womens_shoes,2008-12-27,31
SouthCentral/Tennessee/womens_shoes,2008-12-28,29
SouthCentral/Tennessee/womens_shoes,2008-12-29,28
SouthCentral/Tennessee/womens_shoes,2008-12-30,31
SouthCentral/Tennessee/womens_shoes,2008-12-31,31


time: 10.3 ms (started: 2024-01-04 14:04:20 -03:00)


## 3. Modelagem

In [13]:
#---- Features de data:

from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

@njit
def rolling_mean_7(x):
    return rolling_mean(x, window_size = 7)

@njit
def rolling_mean_14(x):
    return rolling_mean(x, window_size = 14)

@njit
def rolling_mean_21(x):
    return rolling_mean(x, window_size = 21)

@njit
def rolling_mean_28(x):
    return rolling_mean(x, window_size = 28)

time: 6.68 ms (started: 2024-01-04 14:04:20 -03:00)


In [14]:
def rmse(y_true, y_pred):
    
    return np.sqrt(np.mean(np.square(y_true - y_pred)))

time: 754 µs (started: 2024-01-04 14:04:20 -03:00)


In [15]:
n_horizon = Y_valid_df.ds.nunique() # Quantidade de dias para a projeção

time: 6.05 ms (started: 2024-01-04 14:04:20 -03:00)


In [20]:
import optuna
from sklearn.metrics import mean_squared_error

def objective(trial):

    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1)
    num_leaves = trial.suggest_int('num_leaves', 2, 256)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 100)
    bagging_fraction = trial.suggest_float('learning_rate', 1e-3, 1e-1)
    feature_fraction = trial.suggest_float('learning_rate', 1e-3, 1e-1)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-3, 10.0, log = True)
    reg_lambda = trial.suggest_float('reg_alpha', 1e-3, 10.0, log = True)
    min_child_samples = trial.suggest_int('min_child_samples', 5, 100)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    min_child_weight = trial.suggest_float('min_child_weight', 1e-3, 10.0, log = True)
    subsample = trial.suggest_float('subsample', 0.1, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 1.0)
    subsample_freq = trial.suggest_int('subsample_freq', 1, 10)
    min_split_gain = trial.suggest_float('min_split_gain', 1e-4, 1.0, log = True)
    max_delta_step = trial.suggest_int('max_delta_step', 0, 10)
    scale_pos_weight = trial.suggest_float('scale_pos_weight', 0.1, 1.0)
    

    lgbm = LGBMRegressor(num_leaves = num_leaves,
                         max_depth = max_depth, 
                         learning_rate = learning_rate,
                         n_estimators = 500,
                         min_data_in_leaf = min_data_in_leaf, 
                         bagging_fraction = bagging_fraction, 
                         feature_fraction = feature_fraction,
                         reg_alpha = reg_alpha,
                         reg_lambda = reg_lambda,
                         min_child_samples = min_child_samples,
                         min_child_weight = min_child_weight,
                         subsample = subsample,
                         colsample_bytree = colsample_bytree,
                         subsample_freq = subsample_freq,
                         min_split_gain = min_split_gain,
                         max_delta_step = max_delta_step,
                         scale_pos_weight = scale_pos_weight,
                         random_state = 19
                         )
    
    models_list = [lgbm]

    model = MLForecast(models = models_list,
                       freq = 'D',
                       num_threads = 6,
                       lags = [1, 7, 14, 21, 28, 30], 
                       date_features = ['dayofweek', 'month', 'year', 'quarter', 'day', 'week'],
                       lag_transforms = {
                           1: [expanding_mean],
                           7: [rolling_mean_7],
                           14: [rolling_mean_14],
                           21: [rolling_mean_21],
                           28: [rolling_mean_28],
                       }
               )

    model.fit(Y_train_df.reset_index(), id_col = 'unique_id', time_col = 'ds', target_col = 'y', fitted = True)

    Y_hat_df = model.predict(h = n_horizon)

    p = Y_hat_df.reset_index().merge(Y_valid_df.reset_index(), on = ['unique_id', 'ds'], how = 'left')

    error = rmse(p['y'], p['LGBMRegressor'])
    
    return error

time: 1.92 ms (started: 2024-01-04 14:05:12 -03:00)


In [21]:
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 20)

[I 2024-01-04 14:05:13,175] A new study created in memory with name: no-name-6f4f7b5f-6406-4046-9dbb-3783c28232de


[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=83 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] feature_fraction is set=0.05805685598993946, colsample_bytree=0.1606547866531502 will be ignored. Current value: feature_fraction=0.05805685598993946
[LightGBM] [Warning] bagging_fraction is set=0.05805685598993946, subsample=0.5875687293630835 will be ignored. Current value: bagging_fraction=0.05805685598993946
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=83 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] feature_fraction is set=0.05805685598993946, colsample_bytree=0.1606547866531502 will be ignored. Current value: feature_fraction=0.05805685598993946
[LightGBM] [Warning] bagging_fraction is set=0.05805685598993946, subsample=0.5875687293630835 will be ignored. Current value: bagging_fraction=0.05805685598993946
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 

[I 2024-01-04 14:05:20,191] Trial 0 finished with value: 231.30983868415868 and parameters: {'learning_rate': 0.05805685598993946, 'num_leaves': 101, 'min_data_in_leaf': 55, 'reg_alpha': 0.2177333499358141, 'min_child_samples': 83, 'max_depth': 5, 'min_child_weight': 0.08007425289212589, 'subsample': 0.5875687293630835, 'colsample_bytree': 0.1606547866531502, 'subsample_freq': 7, 'min_split_gain': 0.0027001804021674868, 'max_delta_step': 6, 'scale_pos_weight': 0.35293362358087665}. Best is trial 0 with value: 231.30983868415868.


[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=83 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] feature_fraction is set=0.05805685598993946, colsample_bytree=0.1606547866531502 will be ignored. Current value: feature_fraction=0.05805685598993946
[LightGBM] [Warning] bagging_fraction is set=0.05805685598993946, subsample=0.5875687293630835 will be ignored. Current value: bagging_fraction=0.05805685598993946
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=83 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] feature_fraction is set=0.05805685598993946, colsample_bytree=0.1606547866531502 will be ignored. Current value: feature_fraction=0.05805685598993946
[LightGBM] [Warning] bagging_fraction is set=0.05805685598993946, subsample=0.5875687293630835 will be ignored. Current value: bagging_fraction=0.05805685598993946
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=83 will be ignored.

[I 2024-01-04 14:05:27,868] Trial 1 finished with value: 231.08613693157005 and parameters: {'learning_rate': 0.07261204006702895, 'num_leaves': 163, 'min_data_in_leaf': 61, 'reg_alpha': 0.006028638715307247, 'min_child_samples': 20, 'max_depth': 14, 'min_child_weight': 3.03867554259231, 'subsample': 0.685584547587064, 'colsample_bytree': 0.9004943822690571, 'subsample_freq': 9, 'min_split_gain': 0.0012221918428850136, 'max_delta_step': 5, 'scale_pos_weight': 0.9745899484603823}. Best is trial 1 with value: 231.08613693157005.


...

[LightGBM] [Warning] min_data_in_leaf is set=46, min_child_samples=33 will be ignored. Current value: min_data_in_leaf=46
[LightGBM] [Warning] feature_fraction is set=0.035608626868287335, colsample_bytree=0.7452960968224038 will be ignored. Current value: feature_fraction=0.035608626868287335
[LightGBM] [Warning] bagging_fraction is set=0.035608626868287335, subsample=0.5328363136202824 will be ignored. Current value: bagging_fraction=0.035608626868287335
[LightGBM] [Warning] min_data_in_leaf is set=46, min_child_samples=33 will be ignored. Current value: min_data_in_leaf=46
[LightGBM] [Warning] feature_fraction is set=0.035608626868287335, colsample_bytree=0.7452960968224038 will be ignored. Current value: feature_fraction=0.035608626868287335
[LightGBM] [Warning] bagging_fraction is set=0.035608626868287335, subsample=0.5328363136202824 will be ignored. Current value: bagging_fraction=0.035608626868287335
[LightGBM] [Warning] min_data_in_leaf is set=46, min_child_samples=33 will be 

[I 2024-01-04 14:07:56,852] Trial 18 finished with value: 18.413736228833073 and parameters: {'learning_rate': 0.0270466113472588, 'num_leaves': 151, 'min_data_in_leaf': 41, 'reg_alpha': 0.0011272911988604133, 'min_child_samples': 29, 'max_depth': 6, 'min_child_weight': 0.021572671389730787, 'subsample': 0.5427646496114741, 'colsample_bytree': 0.8017418465303501, 'subsample_freq': 2, 'min_split_gain': 0.0018065990197147393, 'max_delta_step': 0, 'scale_pos_weight': 0.1494177242906738}. Best is trial 17 with value: 17.6090018124214.


[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=29 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] feature_fraction is set=0.0270466113472588, colsample_bytree=0.8017418465303501 will be ignored. Current value: feature_fraction=0.0270466113472588
[LightGBM] [Warning] bagging_fraction is set=0.0270466113472588, subsample=0.5427646496114741 will be ignored. Current value: bagging_fraction=0.0270466113472588
[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=29 will be ignored. Current value: min_data_in_leaf=41
[LightGBM] [Warning] feature_fraction is set=0.0270466113472588, colsample_bytree=0.8017418465303501 will be ignored. Current value: feature_fraction=0.0270466113472588
[LightGBM] [Warning] bagging_fraction is set=0.0270466113472588, subsample=0.5427646496114741 will be ignored. Current value: bagging_fraction=0.0270466113472588
[LightGBM] [Warning] min_data_in_leaf is set=41, min_child_samples=29 will be ignored. Current

[I 2024-01-04 14:08:07,981] Trial 19 finished with value: 17.800950167207095 and parameters: {'learning_rate': 0.030117425057549097, 'num_leaves': 150, 'min_data_in_leaf': 39, 'reg_alpha': 0.0010530492066892427, 'min_child_samples': 13, 'max_depth': 6, 'min_child_weight': 0.027530489742442675, 'subsample': 0.6814073910338947, 'colsample_bytree': 0.8021148774253098, 'subsample_freq': 2, 'min_split_gain': 0.0011226124299370558, 'max_delta_step': 0, 'scale_pos_weight': 0.13434101662640205}. Best is trial 17 with value: 17.6090018124214.


[LightGBM] [Warning] min_data_in_leaf is set=39, min_child_samples=13 will be ignored. Current value: min_data_in_leaf=39
[LightGBM] [Warning] feature_fraction is set=0.030117425057549097, colsample_bytree=0.8021148774253098 will be ignored. Current value: feature_fraction=0.030117425057549097
[LightGBM] [Warning] bagging_fraction is set=0.030117425057549097, subsample=0.6814073910338947 will be ignored. Current value: bagging_fraction=0.030117425057549097
[LightGBM] [Warning] min_data_in_leaf is set=39, min_child_samples=13 will be ignored. Current value: min_data_in_leaf=39
[LightGBM] [Warning] feature_fraction is set=0.030117425057549097, colsample_bytree=0.8021148774253098 will be ignored. Current value: feature_fraction=0.030117425057549097
[LightGBM] [Warning] bagging_fraction is set=0.030117425057549097, subsample=0.6814073910338947 will be ignored. Current value: bagging_fraction=0.030117425057549097
[LightGBM] [Warning] min_data_in_leaf is set=39, min_child_samples=13 will be 

In [22]:
study.best_params

{'learning_rate': 0.035608626868287335,
 'num_leaves': 152,
 'min_data_in_leaf': 46,
 'reg_alpha': 0.7493647554395146,
 'min_child_samples': 33,
 'max_depth': 7,
 'min_child_weight': 0.027338398098262277,
 'subsample': 0.5328363136202824,
 'colsample_bytree': 0.7452960968224038,
 'subsample_freq': 4,
 'min_split_gain': 0.00203245703916804,
 'max_delta_step': 0,
 'scale_pos_weight': 0.1703693448857999}

time: 5.45 ms (started: 2024-01-04 14:12:02 -03:00)


In [23]:
#---- Salvando os melhores parâmetros em um JSON:

import json

with open('lgbm-best-parameters.json', 'w') as jsn:
    json.dump(dict(study.best_params), jsn)

time: 2.19 ms (started: 2024-01-04 14:12:02 -03:00)


In [24]:
study.best_value

17.6090018124214

time: 4.51 ms (started: 2024-01-04 14:12:09 -03:00)
